In [ ]:
import sys
sys.path.append("..")

from pathlib import Path

import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.utils.data
import torchvision.transforms as VT
import torchvision.transforms.functional as VF
from torchvision.utils import make_grid

from src.datasets.ca import *

In [ ]:
ds = TotalCADataset(
    (64, 64), 
    seed=23,
    #num_repetitions=8,
    #num_iterations=[1, 20],
    #init_prob=[0, 1],
    #rules=["3-23", "124-45"],
    dtype=torch.uint8,
)
len(ds)

In [ ]:
def plot_images(iterable, num: int = 8, nrow: int = 8):
    images = []
    for image in iterable:
        if isinstance(image, (list, tuple)):
            image = image[0]
        images.append(image * 200)
        if len(images) >= num:
            break
    return VF.to_pil_image(make_grid(images, nrow=nrow))

dl = DataLoader(ds)
plot_images(dl, 256, nrow=16)

In [ ]:
ds2 = TensorDataset(torch.load("../datasets/ca-64x64-i10-p05.pt"))

In [ ]:
class CropDataset(Dataset):
    def __init__(self, dataset: Dataset, shape: Tuple[int, int]):
        self.dataset = dataset
        self.shape = shape
        self.cropper = VT.RandomCrop(self.shape)
        
    def __len(self): return len(self.dataset)

    def __getitem__(self, item):
        item = self.dataset[item]
        if isinstance(item, (list, tuple)):
            return [self.cropper(item[0]).unsqueeze(0), *item[1:]]
        return self.cropper(item).unsqueeze(0)
        
ds3 = CropDataset(ds2, (32, 32))
plot_images(ds3, 256, nrow=16)

In [ ]:
from torch import fft

In [ ]:
def _trans(x):
    f = fft.fft2(x)#[...,:4,:4])
    return (f.real-f.imag) * .01 + .5
    #return VF.resize(f.real, (64, 64)) * .1 + .3# * f.imag)
images = [ds[i][0].unsqueeze(0) for i in range(8)]
images += [_trans(i) for i in images]
VF.to_pil_image(make_grid(images))

In [ ]:
f = fft.fft2(torch.Tensor([[0, 1, 1, 0], [1, 0, 1, 0]]))
f = fft.fft2(torch.Tensor(torch.randn(10, 10)))
f.shape
#torch.cat([f.real, f.imag])#to(torch.float)#imag.shape